In [1]:
import pandas as pd
import numpy as np
import sys
vone_dir = '/user_data/vayzenbe/GitHub_Repos/vonenet'
cornet_dir = '/user_data/vayzenbe/GitHub_Repos/CORnet'

sys.path.insert(1, vone_dir)
sys.path.insert(1, cornet_dir)
import vonenet
import cornet
from torchvision.models import convnext_large, ConvNeXt_Large_Weights, vit_b_16, ViT_B_16_Weights
import torch

import torch.nn as nn
import torchvision
import load_stim
from glob import glob as glob

/home/vayzenbe/anaconda3/envs/ml_new/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
model = cornet.get_model('s', pretrained=True)

In [20]:
layer_call = "getattr(getattr(getattr(model,'module'),'decoder'),'avgpool')"
model_layer = eval(layer_call)

In [18]:
model.module.decoder.avgpool

AdaptiveAvgPool2d(output_size=1)

In [15]:
test = 'cornetsdfdsfds'

test == 'cornets*'

False

In [16]:
'cornet' in test

True

In [2]:
curr_dir = '/user_data/vayzenbe/GitHub_Repos/kornet'
stim_dir = f'{curr_dir}/stim/_things'


layer = ['ln','avgpool','avgpool',['decoder','avgpool']]


In [3]:
cats = glob(f'{stim_dir}/*/')
cats

['/user_data/vayzenbe/GitHub_Repos/kornet/stim/_things/butterfly/',
 '/user_data/vayzenbe/GitHub_Repos/kornet/stim/_things/dog/',
 '/user_data/vayzenbe/GitHub_Repos/kornet/stim/_things/lamp/',
 '/user_data/vayzenbe/GitHub_Repos/kornet/stim/_things/bike/',
 '/user_data/vayzenbe/GitHub_Repos/kornet/stim/_things/truck/',
 '/user_data/vayzenbe/GitHub_Repos/kornet/stim/_things/fish/',
 '/user_data/vayzenbe/GitHub_Repos/kornet/stim/_things/duck/',
 '/user_data/vayzenbe/GitHub_Repos/kornet/stim/_things/bear/',
 '/user_data/vayzenbe/GitHub_Repos/kornet/stim/_things/elephant/',
 '/user_data/vayzenbe/GitHub_Repos/kornet/stim/_things/turtle/',
 '/user_data/vayzenbe/GitHub_Repos/kornet/stim/_things/key/',
 '/user_data/vayzenbe/GitHub_Repos/kornet/stim/_things/squirrel/',
 '/user_data/vayzenbe/GitHub_Repos/kornet/stim/_things/frog/',
 '/user_data/vayzenbe/GitHub_Repos/kornet/stim/_things/hand/',
 '/user_data/vayzenbe/GitHub_Repos/kornet/stim/_things/spoon/',
 '/user_data/vayzenbe/GitHub_Repos/korne

In [13]:
def load_model(model_arch):    
    """
    load model
    """
    if model_arch == 'cornets':
        model = vonenet.get_model(model_arch='cornets', pretrained=False).module
        layer_call = "getattr(getattr(getattr(getattr(model,'module'),'model'),'decoder'),'avgpool')"

    elif model_arch == 'cornets_ff':
        model = vonenet.get_model(model_arch='cornets_ff', pretrained=False).module
        layer_call = "getattr(getattr(getattr(getattr(model,'module'),'model'),'decoder'),'avgpool')"
        

    elif model_arch == 'convnext':
        model = convnext_large(weights=None)
        transform = ConvNeXt_Large_Weights.IMAGENET1K_V1.transforms()
        layer_call = "getattr(getattr(model,'module'),'avgpool')"
    elif model_arch == 'vit':
        model = vit_b_16(weights=None)
        transform = ViT_B_16_Weights.DEFAULT.transforms()
        layer_call = "getattr(getattr(getattr(model,'module'),'encoder'),'ln')"
        layer_call = "getattr(getattr(model, 'encoder'),'ln')"
        #layer_call = "getattr(getattr(getattr(getattr(getattr(getattr(model,'module'),'encoder'),'layers'),'encoder_layer_11'),'mlp'),'3')"


    if model_arch == 'cornets' or model_arch == 'cornets_ff':

        transform = torchvision.transforms.Compose([
                torchvision.transforms.Resize(256),
                torchvision.transforms.CenterCrop(224),
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                 std=[0.5, 0.5, 0.5]),
            ])

    #model = torch.nn.DataParallel(model).cuda()

    
    #checkpoint = torch.load(f'{weights_dir}/{model_arch}_{train_set}_best_1.pth.tar')
    #model.load_state_dict(checkpoint['state_dict'])

    return model, transform, layer_call

In [16]:
def extract_acts(model, image_dir, transform, layer_call):
    print('extracting features...')
    

    #set up hook to specified layer
    def _store_feats(layer, inp, output):
        """An ugly but effective way of accessing intermediate model features
        """
        #avgpool = nn.AdaptiveAvgPool2d(output_size=(1,768))
        #output = avgpool(output)
        

        output = output.cpu().numpy()
        print(output.shape)
        
        _model_feats.append(np.reshape(output, (len(output), -1)))

    try:
        m = model.module
    except:
        m = model
    #model_layer = getattr(getattr(m, layer), sublayer)
    model_layer = eval(layer_call)
    model_layer.register_forward_hook(_store_feats)



    #Iterate through each image and extract activations

    imNum = 0
    n=0

    

    
    test_dataset = load_stim.load_stim(image_dir, transform=transform)
    testloader = torch.utils.data.DataLoader(test_dataset, batch_size=12, shuffle=False, num_workers = 4, pin_memory=True)
    


    with torch.no_grad():
        
        for data, _ in testloader:
            # move tensors to GPU if CUDA is available
            
            #data= data.cuda()
            
            _model_feats = []
            model(data)
            #output = model(data)
            
            out = np.vstack(_model_feats)
            

            if n == 0:
                acts = out
                #label_list = label
            else:
                acts= np.append(acts, out,axis = 0)
                #label_list = np.append(label_list, label)
                
            
            n = n + 1

    return acts

In [3]:
transform = torchvision.transforms.Compose([
                torchvision.transforms.Resize(256),
                torchvision.transforms.CenterCrop(224),
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                 std=[0.5, 0.5, 0.5]),
            ])

image_dir = '/user_data/vayzenbe/GitHub_Repos/kornet/stim/test/Outline'
test_dataset = load_stim.load_stim(image_dir, transform=transform)

['OBJ (01).png', 'OBJ (04).png', 'OBJ (06).png', 'OBJ (07).png', 'OBJ (08).png', 'OBJ (10).png', 'OBJ (11).png', 'OBJ (12).png', 'OBJ (14).png', 'OBJ (16).png', 'OBJ (17).png', 'OBJ (18).png', 'OBJ (19).png', 'OBJ (20).png', 'OBJ (21).png', 'OBJ (22).png', 'OBJ (23).png', 'OBJ (25).png', 'OBJ (27).png', 'OBJ (29).png', 'OBJ (30).png', 'OBJ (31).png', 'OBJ (32).png', 'OBJ (38).png', 'OBJ (41).png', 'OBJ (42).png', 'OBJ (44).png', 'OBJ (45).png', 'OBJ (46).png', 'OBJ (47).png']


In [17]:
cat = 'airplane'
#['vit','convnext','cornets','cornets_ff']
for model_type in ['vit']:
    model, transform, layer_call = load_model(model_type)
    acts = extract_acts(model, f'{stim_dir}/{cat}', transform, layer_call)

    print(acts.shape)


extracting features...
(12, 197, 768)
(12, 151296)


In [19]:
model

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate=none)
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_att

In [31]:
151296/768

197.0

In [40]:
768 * 16

12288

In [6]:
cat = 'airplane'


acts = extract_acts(model, f'{stim_dir}/{cat}', transform)


extracting features...


In [7]:
acts.shape

(12, 512)

In [38]:
try:
    m = model.module
except:
    m = model
getattr(getattr(getattr(getattr(model,'module'),'model'),'decoder'),'avgpool')
#model_layer.register_forward_hook(_store_feats)

AdaptiveAvgPool2d(output_size=1)

In [37]:
getattr(getattr(getattr(getattr(model,'module'),'model'),'decoder'),'avgpool')

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_backward_hooks',
 '_get_item_by_idx',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replic

In [30]:
model

DataParallel(
  (module): Sequential(
    (vone_block): VOneBlock(
      (simple_conv_q0): GFB()
      (simple_conv_q1): GFB()
      (simple): ReLU(inplace=True)
      (complex): Identity()
      (gabors): Identity()
      (noise): ReLU(inplace=True)
      (output): Identity()
    )
    (bottleneck): Conv2d(512, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (model): CORnetSBackEnd(
      (V2): CORblock_S(
        (conv_input): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (skip): Conv2d(128, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (norm_skip): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (nonlin1): ReLU(inplace=True)
        (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (nonlin2): ReLU(inplace=True)
        (conv3): Conv2d(512, 128, kernel_size=(1, 1), stride=

In [27]:
class Person:
    age = 23
    name = "Adam"

person = Person()
print('The age is:', getattr(person, "age"))

The age is: 23


In [28]:
dir(person)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'age',
 'name']

In [9]:
layer = 'decoder'
sublayer = 'avgpool'

#set up hook to specified layer
def _store_feats(layer, inp, output):
    """An ugly but effective way of accessing intermediate model features
    """
    
    output = output.cpu().numpy()
    
    _model_feats.append(np.reshape(output, (len(output), -1)))

try:
    m = model.module
except:
    m = model
model_layer = getattr(getattr(m, layer), sublayer)
model_layer.register_forward_hook(_store_feats)


_model_feats = []
model(im)
model_feats.append(_model_feats[time_step])

In [16]:
nn.Sequential(*list(model.children[0][:,-1]))

TypeError: 'method' object is not subscriptable

In [17]:
model

DataParallel(
  (module): Sequential(
    (vone_block): VOneBlock(
      (simple_conv_q0): GFB()
      (simple_conv_q1): GFB()
      (simple): ReLU(inplace=True)
      (complex): Identity()
      (gabors): Identity()
      (noise): ReLU(inplace=True)
      (output): Identity()
    )
    (bottleneck): Conv2d(512, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (model): CORnetSBackEnd(
      (V2): CORblock_S(
        (conv_input): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (skip): Conv2d(128, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (norm_skip): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (nonlin1): ReLU(inplace=True)
        (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (nonlin2): ReLU(inplace=True)
        (conv3): Conv2d(512, 128, kernel_size=(1, 1), stride=

In [8]:
print(convnext)

ConvNeXt(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((192,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
          (1): Permute()
          (2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=192, out_features=768, bias=True)
          (4): GELU(approximate=none)
          (5): Linear(in_features=768, out_features=192, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
          (1): Permute()
          (2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in